In [25]:
import torch
import torch_pruning as tp
from yolox.models import create_yolox_model

model = create_yolox_model(num_classes=20,name = 'yolox_custom',  
exp_path = '/workspace/tensorrt/YOLOX/exps/default/custom2/type2/all25_stem/yolox_voc_tiny_scratch_all25_stem.py',   # 오리진 모델 불러와서 자르기
ckpt_path = '/workspace/tensorrt/YOLOX/fpgm_2/type2/first/yolox_voc_tiny_all25_stem/35.88.pth'
).cuda()
# 1. setup strategy (L1 Norm)
strategy = tp.strategy.L1Strategy() # or tp.strategy.RandomStrategy()

In [26]:

# 2. build dependency graph for resnet18
DG = tp.DependencyGraph()
DG.build_dependency(model, example_inputs=torch.randn(1,3,224,224).cuda())



In [27]:

temp = []
temp2 = list(model.state_dict().keys())
num_list = [0,1,2,3]
#print(temp2)
for i in range(len(temp2)):
        if 'bn' in temp2[i]:
            continue
        if 'weight' in temp2[i]:
            temp2[i] = temp2[i].replace('.weight','')
            temp2[i] = temp2[i].replace('0.0','0[0]')
            temp2[i] = temp2[i].replace('0.1','0[1]')
            temp2[i] = temp2[i].replace('1.0','1[0]')
            temp2[i] = temp2[i].replace('1.1','1[1]')
            temp2[i] = temp2[i].replace('2.0','2[0]')
            temp2[i] = temp2[i].replace('2.1','2[1]')
            temp2[i] = temp2[i].replace('3.0','3[0]')
            temp2[i] = temp2[i].replace('3.1','3[1]')
            temp2[i] = temp2[i].replace('4.0','4[0]')
            temp2[i] = temp2[i].replace('4.1','4[1]')
            temp2[i] = temp2[i].replace('5.0','5[0]')
            temp2[i] = temp2[i].replace('5.1','5[1]')
            temp2[i] = temp2[i].replace('5.2','5[2]')
            temp2[i] = temp2[i].replace('m.0','m[0]')
            temp2[i] = temp2[i].replace('m.1','m[1]')
            temp2[i] = temp2[i].replace('m.2','m[2]')
            temp.append(temp2[i])

print(temp)
# backbone 만 프루닝


['backbone.backbone.stem.conv.conv', 'backbone.backbone.dark2[0].conv', 'backbone.backbone.dark2[1].conv1.conv', 'backbone.backbone.dark2[1].conv2.conv', 'backbone.backbone.dark2[1].conv3.conv', 'backbone.backbone.dark2[1].m[0].conv1.conv', 'backbone.backbone.dark2[1].m[0].conv2.conv', 'backbone.backbone.dark3[0].conv', 'backbone.backbone.dark3[1].conv1.conv', 'backbone.backbone.dark3[1].conv2.conv', 'backbone.backbone.dark3[1].conv3.conv', 'backbone.backbone.dark3[1].m[0].conv1.conv', 'backbone.backbone.dark3[1].m[0].conv2.conv', 'backbone.backbone.dark3[1].m[1].conv1.conv', 'backbone.backbone.dark3[1].m[1].conv2.conv', 'backbone.backbone.dark3[1].m[2].conv1.conv', 'backbone.backbone.dark3[1].m[2].conv2.conv', 'backbone.backbone.dark4[0].conv', 'backbone.backbone.dark4[1].conv1.conv', 'backbone.backbone.dark4[1].conv2.conv', 'backbone.backbone.dark4[1].conv3.conv', 'backbone.backbone.dark4[1].m[0].conv1.conv', 'backbone.backbone.dark4[1].m[0].conv2.conv', 'backbone.backbone.dark4[1].m

In [28]:
print(model)

YOLOX(
  (backbone): YOLOPAFPN(
    (backbone): CSPDarknet(
      (stem): Focus(
        (conv): BaseConv(
          (conv): Conv2d(12, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn): BatchNorm2d(24, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
      )
      (dark2): Sequential(
        (0): BaseConv(
          (conv): Conv2d(24, 48, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
          (bn): BatchNorm2d(48, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
        (1): CSPLayer(
          (conv1): BaseConv(
            (conv): Conv2d(48, 24, kernel_size=(1, 1), stride=(1, 1), bias=False)
            (bn): BatchNorm2d(24, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
            (act): SiLU(inplace=True)
          )
          (conv2): BaseConv(
            (conv): Conv2d(48, 24, kernel_siz

In [29]:
from pruning_imagenet import Mask

In [30]:
m = Mask(model)
m.mask_index = [24,27,30,33,39,45,54,57,60,63,69,75,90,93,99,102,96  # backbone conv3 30 60 96
                            ,105,108,111,114,117,120,123,141,159] # neck

m.mask_index = [24,27,30,33,39,45,54,57,60,63,69,75,90,93,99,102,96
                            ,105,108,111,114,117,120,123,141,159,126,129,132,135,138,144,147,150,153,156,162,165,168,171,174, #neck
                           ]# 231,234,237, #stems
                           # 177,180,183,186,189,192,195,198,201,204,207,210] # head
        

In [31]:
m.init_length()

In [32]:
m.init_mask(1, 1/6)
print(m.mask_index)

filter codebook done
similar index done
filter codebook done
similar index done
filter codebook done
similar index done
filter codebook done
similar index done
filter codebook done
similar index done
filter codebook done
similar index done
filter codebook done
similar index done
filter codebook done
similar index done
filter codebook done
similar index done
filter codebook done
similar index done
filter codebook done
similar index done
filter codebook done
similar index done
filter codebook done
similar index done
filter codebook done
similar index done
filter codebook done
similar index done
filter codebook done
similar index done
filter codebook done
similar index done
filter codebook done
similar index done
filter codebook done
similar index done
filter codebook done
similar index done
filter codebook done
similar index done
filter codebook done
similar index done
filter codebook done
similar index done
filter codebook done
similar index done
filter codebook done
similar index done


In [33]:
m.do_mask()
m.do_similar_mask()

mask Done
mask similar Done


In [34]:
m.if_zero()
print(m.mask_index)
print(m.similar_matrix)

layer: 24, number of nonzero weight is 2880, zero is 576
layer: 27, number of nonzero weight is 2880, zero is 576
layer: 30, number of nonzero weight is 4320, zero is 864
layer: 33, number of nonzero weight is 1080, zero is 216
layer: 39, number of nonzero weight is 1080, zero is 216
layer: 45, number of nonzero weight is 1080, zero is 216
layer: 54, number of nonzero weight is 11520, zero is 2304
layer: 57, number of nonzero weight is 11520, zero is 2304
layer: 60, number of nonzero weight is 17280, zero is 3456
layer: 63, number of nonzero weight is 4320, zero is 864
layer: 69, number of nonzero weight is 4320, zero is 864
layer: 75, number of nonzero weight is 4320, zero is 864
layer: 90, number of nonzero weight is 46080, zero is 9216
layer: 93, number of nonzero weight is 46080, zero is 9216
layer: 96, number of nonzero weight is 69120, zero is 13824
layer: 99, number of nonzero weight is 17280, zero is 3456
layer: 102, number of nonzero weight is 155520, zero is 31104
layer: 105,

In [35]:
model = m.model


In [36]:
dict ={}


for key, value in model.state_dict().items():
    temp = []
    if len(value.shape) ==4:
        print('pruning channel and layer name : ',len(value),',', key)
        for i in range(len(value)):
            if value[i].sum() == 0: 
                print('pruning channex index : ', i)
                #temp = torch.norm(value.view(len(value), -1) , p= 1, dim=1)
                #print(temp)
                #dict[key].append(i)
                temp.append(i)
    if temp:   
        dict[key]  = temp



pruning channel and layer name :  24 , backbone.backbone.stem.conv.conv.weight
pruning channel and layer name :  48 , backbone.backbone.dark2.0.conv.weight
pruning channel and layer name :  24 , backbone.backbone.dark2.1.conv1.conv.weight
pruning channel and layer name :  24 , backbone.backbone.dark2.1.conv2.conv.weight
pruning channel and layer name :  48 , backbone.backbone.dark2.1.conv3.conv.weight
pruning channel and layer name :  24 , backbone.backbone.dark2.1.m.0.conv1.conv.weight
pruning channel and layer name :  24 , backbone.backbone.dark2.1.m.0.conv2.conv.weight
pruning channel and layer name :  96 , backbone.backbone.dark3.0.conv.weight
pruning channel and layer name :  36 , backbone.backbone.dark3.1.conv1.conv.weight
pruning channex index :  5
pruning channex index :  12
pruning channex index :  13
pruning channex index :  14
pruning channex index :  26
pruning channex index :  31
pruning channel and layer name :  36 , backbone.backbone.dark3.1.conv2.conv.weight
pruning cha

In [37]:
print(dict)
#del dict['backbone.backbone.dark3.1.conv3.conv.weight'][0]

{'backbone.backbone.dark3.1.conv1.conv.weight': [5, 12, 13, 14, 26, 31], 'backbone.backbone.dark3.1.conv2.conv.weight': [1, 2, 3, 6, 14, 34], 'backbone.backbone.dark3.1.conv3.conv.weight': [2, 3, 6, 11, 13, 16, 33, 34, 38, 39, 63, 70], 'backbone.backbone.dark3.1.m.0.conv1.conv.weight': [6, 7, 13, 18, 28, 33], 'backbone.backbone.dark3.1.m.1.conv1.conv.weight': [2, 11, 14, 17, 18, 30], 'backbone.backbone.dark3.1.m.2.conv1.conv.weight': [1, 4, 12, 16, 24, 34], 'backbone.backbone.dark4.1.conv1.conv.weight': [4, 11, 12, 13, 21, 23, 32, 33, 42, 54, 64, 67], 'backbone.backbone.dark4.1.conv2.conv.weight': [6, 11, 13, 15, 26, 47, 49, 55, 56, 58, 68, 69], 'backbone.backbone.dark4.1.conv3.conv.weight': [1, 3, 5, 11, 17, 19, 24, 30, 34, 40, 46, 48, 70, 71, 75, 95, 97, 98, 106, 109, 111, 119, 130, 131], 'backbone.backbone.dark4.1.m.0.conv1.conv.weight': [2, 16, 20, 44, 47, 49, 51, 65, 67, 68, 69, 71], 'backbone.backbone.dark4.1.m.1.conv1.conv.weight': [0, 28, 31, 32, 33, 36, 40, 42, 51, 53, 58, 65]

In [38]:

temp = []
temp2 = list(dict.keys())
for i in range(len(temp2)):
        if 'bn' in temp2[i]:
            continue
        if 'weight' in temp2[i]:
            temp2[i] = temp2[i].replace('.weight','')
            temp2[i] = temp2[i].replace('.0.0','[0][0]')
            temp2[i] = temp2[i].replace('.0.1','[0][1]')
            temp2[i] = temp2[i].replace('.1.0','[1][0]')
            temp2[i] = temp2[i].replace('.1.1','[1][1]')
            temp2[i] = temp2[i].replace('.2.0','[2][0]')
            temp2[i] = temp2[i].replace('.2.1','[2][1]')
            temp2[i] = temp2[i].replace('3.0','3[0]')
            temp2[i] = temp2[i].replace('3.1','3[1]')
            temp2[i] = temp2[i].replace('4.0','4[0]')
            temp2[i] = temp2[i].replace('4.1','4[1]')
            temp2[i] = temp2[i].replace('5.0','5[0]')
            temp2[i] = temp2[i].replace('5.1','5[1]')
            temp2[i] = temp2[i].replace('5.2','5[2]')
            temp2[i] = temp2[i].replace('m.0','m[0]')
            temp2[i] = temp2[i].replace('m.1','m[1]')
            temp2[i] = temp2[i].replace('m.2','m[2]')
            temp2[i] = temp2[i].replace('.0','[0]')
            temp2[i] = temp2[i].replace('.1','[1]')
            temp2[i] = temp2[i].replace('.2','[2]')
            temp.append(temp2[i])

#temp.insert(27,'head.stems[0].conv')
#temp.insert(28,'head.stems[1].conv')
#temp.insert(29,'head.stems[2].conv')

import copy
print(temp)
key = []
temp2 = copy.deepcopy(temp)
for i in range(len(temp2)):
      
            temp2[i] = temp2[i].replace('[0][0]','.0.0')
            temp2[i] = temp2[i].replace('[0][1]','.0.1',)
            temp2[i] = temp2[i].replace('[1][0]','.1.0',)
            temp2[i] = temp2[i].replace('[1][1]','.1.1',)
            temp2[i] = temp2[i].replace('[2][0]','.2.0',)
            temp2[i] = temp2[i].replace('[2][1]','.2.1',)
            temp2[i] = temp2[i].replace('3[0]','3.0',)
            temp2[i] = temp2[i].replace('3[1]','3.1')
            temp2[i] = temp2[i].replace('4[0]','4.0',)
            temp2[i] = temp2[i].replace('4[1]','4.1',)
            temp2[i] = temp2[i].replace('5[0]','5.0',)
            temp2[i] = temp2[i].replace('5[1]','5.1',)
            temp2[i] = temp2[i].replace('5[2]','5.2',)
            temp2[i] = temp2[i].replace('m[0]','m.0',)
            temp2[i] = temp2[i].replace('m[1]','m.1',)
            temp2[i] = temp2[i].replace('m[2]','m.2',)
            temp2[i] = temp2[i].replace('[0]','.0',)
            temp2[i] = temp2[i].replace('[1]','.1',)
            temp2[i] = temp2[i].replace('[2]','.2',)
            key.append(temp2[i])

print(key)
print(temp)

['backbone.backbone.dark3[1].conv1.conv', 'backbone.backbone.dark3[1].conv2.conv', 'backbone.backbone.dark3[1].conv3.conv', 'backbone.backbone.dark3[1].m[0].conv1.conv', 'backbone.backbone.dark3[1].m[1].conv1.conv', 'backbone.backbone.dark3[1].m[2].conv1.conv', 'backbone.backbone.dark4[1].conv1.conv', 'backbone.backbone.dark4[1].conv2.conv', 'backbone.backbone.dark4[1].conv3.conv', 'backbone.backbone.dark4[1].m[0].conv1.conv', 'backbone.backbone.dark4[1].m[1].conv1.conv', 'backbone.backbone.dark4[1].m[2].conv1.conv', 'backbone.backbone.dark5[2].conv1.conv', 'backbone.backbone.dark5[2].conv2.conv', 'backbone.backbone.dark5[2].conv3.conv', 'backbone.backbone.dark5[2].m[0].conv1.conv', 'backbone.backbone.dark5[2].m[0].conv2.conv', 'backbone.lateral_conv0.conv', 'backbone.C3_p4.conv1.conv', 'backbone.C3_p4.conv2.conv', 'backbone.C3_p4.conv3.conv', 'backbone.C3_p4.m[0].conv1.conv', 'backbone.C3_p4.m[0].conv2.conv', 'backbone.reduce_conv1.conv', 'backbone.C3_p3.conv1.conv', 'backbone.C3_p3.c

In [39]:
value = list(dict.values())
value

[[5, 12, 13, 14, 26, 31],
 [1, 2, 3, 6, 14, 34],
 [2, 3, 6, 11, 13, 16, 33, 34, 38, 39, 63, 70],
 [6, 7, 13, 18, 28, 33],
 [2, 11, 14, 17, 18, 30],
 [1, 4, 12, 16, 24, 34],
 [4, 11, 12, 13, 21, 23, 32, 33, 42, 54, 64, 67],
 [6, 11, 13, 15, 26, 47, 49, 55, 56, 58, 68, 69],
 [1,
  3,
  5,
  11,
  17,
  19,
  24,
  30,
  34,
  40,
  46,
  48,
  70,
  71,
  75,
  95,
  97,
  98,
  106,
  109,
  111,
  119,
  130,
  131],
 [2, 16, 20, 44, 47, 49, 51, 65, 67, 68, 69, 71],
 [0, 28, 31, 32, 33, 36, 40, 42, 51, 53, 58, 65],
 [10, 16, 17, 23, 31, 43, 52, 56, 59, 64, 67, 71],
 [1,
  12,
  20,
  24,
  29,
  30,
  43,
  45,
  55,
  57,
  59,
  60,
  72,
  75,
  78,
  83,
  105,
  106,
  109,
  112,
  122,
  124,
  138,
  140],
 [3,
  4,
  7,
  8,
  9,
  10,
  11,
  20,
  44,
  47,
  59,
  65,
  66,
  70,
  75,
  78,
  83,
  88,
  95,
  102,
  103,
  111,
  123,
  138],
 [4,
  16,
  27,
  35,
  37,
  42,
  50,
  55,
  56,
  60,
  63,
  64,
  91,
  98,
  100,
  106,
  108,
  122,
  129,
  137,
  145,

In [40]:
for i in range(len(temp)):
    pruning_plan = DG.get_pruning_plan(eval(f"model.{temp[i]}"), tp.prune_conv_out_channel, dict[(key[i]) + '.weight'])
    print(pruning_plan)
    if DG.check_pruning_plan(pruning_plan):
        pruning_plan.exec()     

Conv2d(96, 36, kernel_size=(1, 1), stride=(1, 1), bias=False)

--------------------------------
          Pruning Plan
--------------------------------
User pruning:
[ [DEP] ConvOutChannelPruner on backbone.backbone.dark3.1.conv1.conv (Conv2d(96, 36, kernel_size=(1, 1), stride=(1, 1), bias=False)) => ConvOutChannelPruner on backbone.backbone.dark3.1.conv1.conv (Conv2d(96, 36, kernel_size=(1, 1), stride=(1, 1), bias=False)), Index=[5, 12, 13, 14, 26, 31], metric={'#params': 576}]

Coupled pruning:
[ [DEP] ConvOutChannelPruner on backbone.backbone.dark3.1.conv1.conv (Conv2d(96, 36, kernel_size=(1, 1), stride=(1, 1), bias=False)) => BatchnormPruner on backbone.backbone.dark3.1.conv1.bn (BatchNorm2d(36, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)), Index=[5, 12, 13, 14, 26, 31], metric={'#params': 12}]
[ [DEP] BatchnormPruner on backbone.backbone.dark3.1.conv1.bn (BatchNorm2d(36, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)) => ElementWiseOpPruner o

In [41]:
print(model)

YOLOX(
  (backbone): YOLOPAFPN(
    (backbone): CSPDarknet(
      (stem): Focus(
        (conv): BaseConv(
          (conv): Conv2d(12, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn): BatchNorm2d(24, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
      )
      (dark2): Sequential(
        (0): BaseConv(
          (conv): Conv2d(24, 48, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
          (bn): BatchNorm2d(48, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
        (1): CSPLayer(
          (conv1): BaseConv(
            (conv): Conv2d(48, 24, kernel_size=(1, 1), stride=(1, 1), bias=False)
            (bn): BatchNorm2d(24, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
            (act): SiLU(inplace=True)
          )
          (conv2): BaseConv(
            (conv): Conv2d(48, 24, kernel_siz

In [42]:
ckpt_state = {
                "model": model.state_dict(),
            }

torch.save(ckpt_state, 'type2_0.375_all_v1.pth')


